# Part II. Graded.

**Due date december 1st**
You can also send your answers to the optional questions 

1. Based on the minimal code example with validation and $L_2$ prenality, implement your own version of a factorization machine. As we do want to use fields this is the same than a rank k asumption on the matrix of the second order terms which can be expressed as $<v_i, v_j>$ with $v_i \in \mathbb{R}^k$ .  Remind that 
$$
\frac{\partial}{\partial \theta} \hat{y}(\mathbf{x})= \begin{cases}1, & \text { if } \theta \text { is } \mbox{the bias term } \omega_0 \\ x_{i}, & \text { if } \theta \text { is } w_{i} \mbox{ (first order term)} \\ x_{i} \sum_{j=1}^{n} v_{j, f} x_{j}-v_{i, f} x_{i}^{2}, & \text { if } \theta \text { is the } i,f \mbox { second order term } <v_i,v_f>   \end{cases}
$$

One possibility to code it is to isolate the second order term within a second vector of parameters while performing the ```get_x``` and change the way the gradient is computed in the ```update``` function. Details about factorization machines are in the [original paper](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf).

In [ ]:
!wget http://go.criteo.net/criteo-ppml-challenge-adkdd21-dataset.zip
!unzip criteo-ppml-challenge-adkdd21-dataset.zip
!rm aggregated_noisy_data_pairs.csv.gz aggregated_noisy_data_singles.csv.gz  
!gunzip X_test.csv.gz X_train.csv.gz y_test.csv.gz y_train.csv.gz
!rm criteo-ppml-challenge-adkdd21-dataset.zip
!sed -n 1,10000p X_test.csv > X_valid.csv #CARE we took 10k lines from test as validation.
!sed -n 1,10000p y_test.csv > y_valid.csv

--2022-01-14 14:33:43--  http://go.criteo.net/criteo-ppml-challenge-adkdd21-dataset.zip
Resolving go.criteo.net (go.criteo.net)... 178.250.0.152
Connecting to go.criteo.net (go.criteo.net)|178.250.0.152|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://criteostorage.blob.core.windows.net/criteo-research-datasets/criteo_ppml_challenge_adkdd2021/criteo-ppml-challenge-adkdd21-dataset.zip [following]
--2022-01-14 14:33:44--  https://criteostorage.blob.core.windows.net/criteo-research-datasets/criteo_ppml_challenge_adkdd2021/criteo-ppml-challenge-adkdd21-dataset.zip
Resolving criteostorage.blob.core.windows.net (criteostorage.blob.core.windows.net)... 20.209.1.1
Connecting to criteostorage.blob.core.windows.net (criteostorage.blob.core.windows.net)|20.209.1.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 297834690 (284M) [application/zip]
Saving to: ‘criteo-ppml-challenge-adkdd21-dataset.zip’

l-challenge-adkdd21   4%[         

## Answer question 1 : 

To answer that question we introduce the matrix ```v``` that has a ```(D,K)``` shape. After that we modified the ```get_p``` and ```update_w``` functions : 
- ```get_p_factorization``` know that into account the $\sum_{i=1}^{n}\sum_{j=i+1}^{n} x_i x_j v_{i, j}$ part.
- ```update_wL2_factorization``` update the values for the matrix ```v``` with an L2 regularization.

In [2]:
# ANSWER - same stuff with a L2 regularizer (bias term + quad terms)
# A lot of settings allows a large difference between train and validation, theses are indeed bad
# Remark than combining L1 and L2 is easy and is often named "elasticnet"
import matplotlib.pyplot as plt
import math


# ANSWER with ony one vector for parameters. This is also possible to split the first and second order terms
def get_x(csv_row, D):
    x = []
    # normal features
    for key, value in csv_row.items():
        index = int(value + key[4:], 16) % D
        x.append(index)
    x.append(0)
    return x


def compute_validation_loss_factorization(w, D, v, k):
    ### TODO Complete this function ###
    # ANSWER
    val_loss = 0
    for t, (row, y) in enumerate(zip(DictReader(open(X_valid)), DictReader(open(y_valid)))):
        x = get_x(row, D)
        p = get_p_factorization(x, w, v, k)
        target = float(y['click'])
        val_loss += logloss(p, target)
    return val_loss / t


def get_p_factorization(x, w, v, k):
    wTx = 0.
    for i in x:
        # wTx
        wTx += w[i] * 1.  # w[i] * 1

        # <v_i, v_j>*(x_i * x_j)

    for f in range(k):
        sv = np.sum(v[x, f] * 1) ** 2
        wTx += 0.5 * (sv - np.sum(v[x, f] ** 2 * 1))

    return 1. / (1. + exp(-max(min(wTx, 20.), -20.)))  # bounded sigmoid


def update_wL2_factorization(w, v, n, x, p, y, k, lbd=5e-4, alpha=.01):
    for index, value in enumerate(x):
        # Update w 
        w[value] -= ((p - y) + lbd * 2 * w[value]) * alpha / (sqrt(n[value]) + 1.)
        n[value] += 1.
    # Update v with L2 regularization
    for f in range(k):
        sv = np.sum(v[x, f])
        for index, value in enumerate(x):
            v[value, f] -= ((p - y) * (sv - v[value, f]) + lbd * 2 * v[value, f]) * alpha

    return w, v, n


In [3]:
D = 2 ** 24
w = [0.] * D  
n = [0.] * D
loss = 0.
n_epochs = 1
n_updates = 0
training_losses = [] 
validation_losses = [] 
alpha = .01 
k=5

# Initialisation de v
import numpy as np
v = np.full(shape = (D, k), fill_value = -0.01)



for e in range(n_epochs):
  training_loss = 0
  for t, (row, y)  in enumerate(zip(DictReader(open(X_opt)), DictReader(open(y_opt)))):
      x = get_x(row, D)  # I gather the memory placement of the coefficients
      p = get_p_factorization(x, w, v, k) # I gather the prediction
      target = float(y['click'])
      training_loss += logloss(p, target) # I compare my model
      if n_updates% 10000 == 0 and n_updates>1:
          training_losses.append( training_loss/t )
          validation_losses.append( compute_validation_loss_factorization(w, D, v, k) )
          print('%s\tupdates: %d\tcurrent logloss on train: %f\tcurrent logloss on validation: %f' % (
              datetime.now(), n_updates, training_losses[-1], validation_losses[-1] ))
      w, v, n = update_wL2_factorization(w, v, n, x, p, target, k) # Upatede the coefficients  
      n_updates += 1

x = [10000*i for i in range(len(training_losses))]
plt.plot(x, training_losses, label='Train')
plt.plot(x, validation_losses, label='Validation')
plt.xlabel('Number of updates')
plt.ylabel('Log Loss')
plt.legend( ('Train', 'Validation') )
plt.show() 

wL2 = w #keep it for later use

#Depending on the learning rate and the coefficent in front of regularization this is possible 
#to have some overfitting advocating for early stopping. Remark that it does not hurt as hard the validation performance 
#Gains remains quite unclear, possibly due to the small dataset size   

NameError: name 'DictReader' is not defined